In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.preprocessing import image, image_dataset_from_directory

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from PIL import Image
from skimage import data, io, filters
from keras.losses import *
from keras.metrics import *
from tensorflow.keras.applications import EfficientNetB7, inception_v3

In [3]:
dir_path = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/'

In [4]:
data = []
!cp -r "../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT" "./"
os.makedirs("./images") 
os.makedirs("./masks") 

SHAPE = (250,250)
for dirname, _, filenames in os.walk("./Dataset_BUSI_with_GT"):
    if "normal" not in dirname:
        for filename in filenames:
            if "mask" not in filename:
                d1 = dirname.split('/')[2]
                d2 = os.path.join('./images', filename)
                d3 = os.path.join('./masks',filename)
                os.replace(dirname+"/"+filename.replace('.png','') + ("_mask.png"), "./masks/" + filename)
                os.replace(dirname+"/"+filename, "./images/" + filename)

                data.append([d1,d2,d3])
                
df = pd.DataFrame(data, columns=['category','image','mask'])
df

In [5]:
IMAGE_SIZE = (300,300)

for i in df.itertuples():
    img = io.imread(i.image)
    mask = io.imread(i.mask)

In [6]:
IMAGE_SIZE = (250,250)

n=5
samples = df.sample(n=n, random_state=1)
print(samples.columns)
print(type(samples))

fig, axs = plt.subplots(n,3,figsize=(n*5,n*10))

for idx,i in enumerate(samples.itertuples()):
    mask = tf.io.read_file(i.mask)
    img = tf.io.read_file(i.image)
    
    img = tf.image.decode_jpeg(img, channels=1)
    mask = tf.image.decode_jpeg(mask, channels=1)
    
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    mirror = (mask+1)%2
    reshaped = tf.image.resize(img, size=[SHAPE[0],SHAPE[1]])    
    axs[idx,0].set_title(str(img.shape))
    axs[idx,0].imshow(img)
    axs[idx,1].imshow(mask)
    axs[idx,2].set_title(str("mirror"))
    axs[idx,2].imshow(mirror)

In [9]:
def process_img(file_path,shape=SHAPE,ismask=False):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=[shape[0],shape[1]])
    if ismask:
        mirror = (img+1)%2
        return tf.concat([img,mirror],2)
    return img


In [25]:
class U_Net():
    def __init__(self, inp=(572, 572, 1),min_filter=64,max_filter=1024, optimizer='Adam',loss= 'BinaryCrossentropy' , metrics = ['accuracy',AUC()]):
        self.max_filter = max_filter
        self.min_filter = min_filter
        self.img_inp = Input(shape = inp)
        down,cac,down_list = self.downsample()
        lower,down_list = self.contracting_path(down,down_list)
        upper,cac2 = self.upsample(lower,down_list)
        out = Conv2D(2,(1), activation="softmax", name='output')(upper)
        self.out = out
        self.model = keras.Model(inputs=self.img_inp, outputs=self.out)
        
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=0.01, momentum=0.9,  name="SGD"
        )
        self.model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
        
    def downsample(self):
        k = self.min_filter

        down_list = []
        res = self.img_inp
        while k < self.max_filter:
            res = Conv2D(k,(3,3))(res)
            res = ReLU()(res)
            #rest = Dropout(0.3)(res)
            res = Conv2D(k,(3,3))(res)
            res = ReLU()(res)
            #rest = Dropout(0.3)(res)
            down_list.append(res)
            cac = res
            res = MaxPooling2D(pool_size=(2, 2),strides=2)(res)
            curr = res
            k*=2
        return res,cac,down_list
    
    def upsample(self,res,down_list):
        k = self.max_filter
        i=0
        while k > self.min_filter:
            res = Conv2D(k/2,(3,3))(res)
            res = ReLU()(res)
            #rest = Dropout(0.3)(res)
            k/=2
            
            if k > self.min_filter:
                res = Conv2D(k/2,(3,3))(res)
                res = ReLU()(res)
                #rest = Dropout(0.3)(res)
                res = UpSampling2D((2,2))(res)
                left = down_list.pop()
                res = self.crop_and_conc(left,res)
                i+=1
            else:
                res = Conv2D(k,(3,3))(res)
                res = ReLU()(res)
                #rest = Dropout(0.3)(res)
            cac = res
            curr = res
        return res,cac
    
    def contracting_path(self,down,down_list):
        res = Conv2D(self.max_filter,(3,3))(down)
        res = ReLU()(res)
        #rest = Dropout(0.3)(res)
        res = Conv2D(self.max_filter/2,(3,3))(res)
        res = ReLU()(res)
        #rest = Dropout(0.3)(res)
        res = UpSampling2D((2,2))(res)
        left = down_list.pop()
        res = self.crop_and_conc(left,res)
        return res,down_list
    
    
    def crop_and_conc(self,left,right):
        left = Cropping2D(cropping=int((left.shape[1]-right.shape[1])/2))(left)
        right = Concatenate()([left,right])
        return right
 

In [26]:
shape = (300,300,1) #tuple for building the model

model = U_Net(inp=shape)
model.model.summary()

#dot_img_file = 'model.png'
#tf.keras.utils.plot_model(model.model, to_file=dot_img_file, show_shapes=False)

In [27]:
from glob import glob

x_files = glob('./images/*.png')
y_files = glob('./masks/*.png')
files_ds = tf.data.Dataset.from_tensor_slices((x_files, y_files))
mask_shape = model.model.get_layer('output').output_shape
print(mask_shape[1:])
files_ds = files_ds.map(lambda x, y: (process_img(x,shape), process_img(y,mask_shape[1:],ismask=True))).batch(1)

print(type(files_ds))

In [ ]:
batch_size = 32
LR = 1e-2
epochs = 75
history = model.model.fit(files_ds,epochs=epochs, batch_size=batch_size)

In [ ]:
plt.title("Reconstruction Loss")
epochs=len(history.history['loss'])
plt.plot(np.arange(0,epochs),history.history['loss'],label="Loss on Train Data", marker='*',color='green')

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,epochs,5),np.arange(0,epochs,5))
plt.legend()
plt.show()
